Cell 1: Install Dependencies

In [ ]:
!pip install -q transformers accelerate peft datasets trl bitsandbytes


Cell 2: Import Packages

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset
import torch


Cell 3: Load LLaMA 13B base model + tokenizer

In [ ]:
base_model = "meta-llama/Llama-2-13b-hf"

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_4bit=True,
    device_map="auto"
)
model = prepare_model_for_kbit_training(model)


Cell 4: Apply LoRA Adapter

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)


Cell 5: Load Your Dataset

In [ ]:
dataset = load_dataset("json", data_files="data/fin_llm_instruct.jsonl")


Cell 6: Format for Chat-style Instruction Tuning

In [ ]:
def format_instruction(example):
    return {
        "input_ids": tokenizer(
            f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['response']}",
            truncation=True, padding="max_length", max_length=512, return_tensors="pt"
        )["input_ids"].squeeze()
    }

tokenized_dataset = dataset["train"].map(format_instruction)


Cell 7: Set Training Arguments & Start Training

In [ ]:
training_args = TrainingArguments(
    output_dir="fin-llm-llama13b",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_dir="./logs",
    save_strategy="epoch"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    tokenizer=tokenizer,
    max_seq_length=512
)

trainer.train()


Cell 8: Save Fine-Tuned Model

In [ ]:
model.save_pretrained("models/financial_llm_llama13b_lora")
tokenizer.save_pretrained("models/financial_llm_llama13b_lora")
